# RCNN Baseline Model

## Preprocessing Dataset

### ESD Dataset

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import pickle

c:\Users\iamva\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# classes= ['Agriculture','Beach','Forest','Residential','River','Seawater']
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2]
)

train_dir = '../ESD'
img_height, img_width = 128, 128
batch_size = 32
num_classes = 6

train_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
print(train_set.class_indices)

Found 3750 images belonging to 6 classes.
{'Agricluture': 0, 'Beach': 1, 'Forest': 2, 'Residential': 3, 'River': 4, 'Seawater': 5}


### WSD Dataset

In [3]:
test_datagen = ImageDataGenerator(
    rescale=1.0/255
)
test_dir = '../Validation'
img_height, img_width = 128, 128
batch_size = 32
num_classes = 6
test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
print(test_set.class_indices)

Found 365 images belonging to 6 classes.
{'Agriculture': 0, 'Beach': 1, 'Forest': 2, 'Residential': 3, 'River': 4, 'Seawater': 5}


## Model Architecture - RCNN

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, Flatten, Dense, Dropout, Reshape

model = Sequential()

# Convolutional Blocks
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Reshape for LSTM
model.add(Flatten())  
model.add(Dense(128))  
# Reshape to have a single timestep for each sample
model.add(Reshape((1, 128)))

# Recurrent Layer (LSTM)
model.add(LSTM(128, return_sequences=True))

# Dense Layers
model.add(Flatten())  # Flatten the LSTM output
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(6, activation='softmax'))  # 6 classes for land cover types

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 57600)             0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

## Model Architecture - ResNet50

In [5]:
# Load pre-trained ResNet50 model without the top classification layer
resnet_model = ResNet50(
    weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [6]:
# Freeze the layers in the ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

# Add custom classification layers on top of ResNet50
x = resnet_model.output
x = GlobalAveragePooling2D()(x)  # Convert features to vectors
x = Dense(128, activation='relu')(x)  # Dense layer with 128 units
x = Dropout(0.2)(x)  # Dropout layer for regularization
predictions = Dense(6, activation='softmax')(x)  # Output layer with 6 classes

# Create the final model
model = Model(inputs=resnet_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 134, 134, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 64, 64, 64)           9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 64, 64, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                         

### Training & Testing

In [11]:
e= 30 #epochs
history = model.fit(esd_set, epochs=e, validation_data=wsd_set)
test_loss, test_acc = model.evaluate(wsd_set)
print('Test Accuracy: ', test_acc)
# Epoch 30/50
# 77/129 [================>.............] - ETA: 52s - loss: 0.8158 - accuracy: 0.7332

Epoch 1/30
129/129 [==============================] - 208s 2s/step - loss: 1.5182 - accuracy: 0.4068 - val_loss: 1.5156 - val_accuracy: 0.4304
Epoch 2/30
129/129 [==============================] - 164s 1s/step - loss: 1.3932 - accuracy: 0.4498 - val_loss: 1.4761 - val_accuracy: 0.4799
Epoch 3/30
129/129 [==============================] - 161s 1s/step - loss: 1.3327 - accuracy: 0.4627 - val_loss: 1.4653 - val_accuracy: 0.5332
Epoch 4/30
129/129 [==============================] - 161s 1s/step - loss: 1.2720 - accuracy: 0.4887 - val_loss: 1.4683 - val_accuracy: 0.5585
Epoch 5/30
129/129 [==============================] - 161s 1s/step - loss: 1.2416 - accuracy: 0.4960 - val_loss: 1.4514 - val_accuracy: 0.5524
Epoch 6/30
129/129 [==============================] - 161s 1s/step - loss: 1.2212 - accuracy: 0.5193 - val_loss: 1.4325 - val_accuracy: 0.5747
Epoch 7/30
129/129 [==============================] - 160s 1s/step - loss: 1.1837 - accuracy: 0.5356 - val_loss: 1.4374 - val_accuracy: 0.5682

In [12]:
model.save('resnet_model')

INFO:tensorflow:Assets written to: resnet_model\assets


INFO:tensorflow:Assets written to: resnet_model\assets


In [9]:
# Train the model
history = model.fit(
    esd_set,
    steps_per_epoch=esd_set.n//esd_set.batch_size,
    epochs=20,
    validation_data=wsd_set,
    validation_steps=wsd_set.n//wsd_set.batch_size
)

Epoch 1/30
117/117 [==============================] - 95s 809ms/step - loss: 1.2819 - accuracy: 0.4801 - val_loss: 1.2840 - val_accuracy: 0.4830
Epoch 2/30
117/117 [==============================] - 88s 751ms/step - loss: 1.2298 - accuracy: 0.5062 - val_loss: 1.2219 - val_accuracy: 0.5682
Epoch 3/30
117/117 [==============================] - 87s 738ms/step - loss: 1.2189 - accuracy: 0.5145 - val_loss: 1.1670 - val_accuracy: 0.6364
Epoch 4/30
117/117 [==============================] - 88s 746ms/step - loss: 1.1823 - accuracy: 0.5218 - val_loss: 1.1647 - val_accuracy: 0.6420
Epoch 5/30
117/117 [==============================] - 87s 742ms/step - loss: 1.1606 - accuracy: 0.5422 - val_loss: 1.1385 - val_accuracy: 0.6420
Epoch 6/30
117/117 [==============================] - 87s 746ms/step - loss: 1.1590 - accuracy: 0.5460 - val_loss: 1.1481 - val_accuracy: 0.5739
Epoch 7/30
117/117 [==============================] - 93s 791ms/step - loss: 1.1330 - accuracy: 0.5503 - val_loss: 1.0944 - val_ac

In [13]:
with open('res50_model','wb') as f:
    pickle.dump(model,f)

In [14]:
with open('res50_model', 'rb') as f:
    res50_model= pickle.load(f)

In [20]:
image = Image.open('test6.jpeg')
image = image.resize((128, 128))
image_array = np.array(image)
image_array = image_array / 255.0

image_array = np.expand_dims(image_array, axis=0)
predictions = res50_model.predict(image_array)

# Get the predicted class
predicted_class = np.argmax(predictions, axis=1)[0]
print(predictions)
print("Predicted Class: ", predicted_class)
print(classes)

1/1 [==============================] - 0s 82ms/step
[[0.36256045 0.02703528 0.1019417  0.00186121 0.05062217 0.45597914]]
Predicted Class:  5
['Agriculture', 'Beach', 'Forest', 'Residential', 'River', 'Seawater']
